# 2023-08-25: AutoML Regression Experiments

## Authors

* Kevin Chu (kevin@velexi.com)

## Overview

This Jupyter notebook demonstrates the use of AutoML to quickly assess multiple common ML regression models for a toy problem. It is based on the tutorial provided by PyCaret at https://www.pycaret.org/tutorials/html/REG101.html.

## History

### 2023-08-25

- Initial version of notebook.

## Experimentation & Development

### Imports

In [1]:
# --- Imports

# External packages
from pycaret import regression
from pycaret.datasets import get_data

### Parameters

In [2]:
# Dataset
dataset_name = "diamond"

# AutoML
experiment_name = "automl-regression-test"
num_best_models = 7
random_seed = 123  # seed used for random number generators to ensure reproducibility of results in this notebook

### Prepare Data

In [3]:
# --- Load dataset

data_df = get_data('diamond')

# --- Check DataFrame

print(f"Number of records: {len(data_df.index)}")
print(f"Columns: {list(data_df.columns)}")

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


Number of records: 6000
Columns: ['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


In [4]:
# --- Construct hold-out dataset

data_df = data_df.sample(frac=0.9, random_state=random_seed).reset_index(drop=True)
data_unseen_df = data_df.drop(data_df.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data_df.shape))
print('Unseen Data For Evaluation of Tuned Model: ' + str(data_unseen_df.shape))

Data for Modeling: (5400, 8)
Unseen Data For Evaluation of Tuned Model: (0, 8)


### Perform AutoML Evaluation

In [5]:
# --- Perform AutoML Evaluation

# Set up the dataset for AutoML regression
regression.setup(data=data_df,
                 target="Price",
                 log_experiment=True,
                 experiment_name=experiment_name,
                 session_id=random_seed,
                ) 

# Automatically train, test, and evaluate models
best_models = regression.compare_models(n_select=num_best_models, verbose=False)

,Description,Value
0,Session id,123
1,Target,Price
2,Target type,Regression
3,Original data shape,"(5400, 8)"
4,Transformed data shape,"(5400, 29)"
5,Transformed train set shape,"(3779, 29)"
6,Transformed test set shape,"(1621, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


### Analyze Results

In [6]:
# Best models
for model in best_models:
    print(model)
    print()

ExtraTreesRegressor(n_jobs=-1, random_state=123)

RandomForestRegressor(n_jobs=-1, random_state=123)

GradientBoostingRegressor(random_state=123)

LGBMRegressor(n_jobs=-1, random_state=123)

DecisionTreeRegressor(random_state=123)

Ridge(random_state=123)

Lasso(random_state=123)



In [7]:
# Display score table
regression.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,757.9822,2.272753e+06,1476.2494,0.9783,0.0798,0.0600,0.056
rf,Random Forest Regressor,763.2111,2.656472e+06,1582.1941,0.9747,0.0793,0.0592,0.055
gbr,Gradient Boosting Regressor,941.2766,3.516252e+06,1844.8599,0.9664,0.1026,0.0782,0.051
lightgbm,Light Gradient Boosting Machine,780.8216,3.561541e+06,1834.3033,0.9660,0.0788,0.0569,0.050
dt,Decision Tree Regressor,980.8632,4.869726e+06,2151.9240,0.9530,0.1046,0.0761,0.049
ridge,Ridge Regression,2416.2896,1.423390e+07,3757.7832,0.8622,0.6110,0.2797,0.048
lasso,Lasso Regression,2411.9718,1.424823e+07,3757.7641,0.8620,0.6103,0.2788,0.095
llar,Lasso Least Angle Regression,2411.9918,1.424821e+07,3757.7623,0.8620,0.6103,0.2788,0.049
br,Bayesian Ridge,2415.4019,1.426339e+07,3760.1952,0.8618,0.6133,0.2795,0.049
lr,Linear Regression,2532.9422,1.551599e+07,3908.8662,0.8494,0.6744,0.2940,0.172


### Tune Promising Models

In [8]:
# --- Extra Trees Regressor

et_model = regression.create_model('et')
et_model_tuned = regression.tune_model(et_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,656.8496,1385351.4689,1177.0095,0.9843,0.0735,0.0568
1,695.6708,1482894.3498,1217.7415,0.9840,0.0824,0.0599
2,756.1282,1648361.4215,1283.8853,0.9860,0.0763,0.0590
3,746.1383,1798134.3896,1340.9453,0.9820,0.0830,0.0610
4,832.3255,3552837.2282,1884.8971,0.9660,0.0815,0.0614
5,872.6804,4307289.3929,2075.4010,0.9645,0.0825,0.0616
6,797.7760,2998419.4551,1731.5945,0.9709,0.0846,0.0630
7,636.9389,1252903.4129,1119.3317,0.9865,0.0752,0.0574
8,792.4113,2020576.8931,1421.4700,0.9805,0.0785,0.0594


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,811.5720,2562415.8994,1600.7548,0.9709,0.0897,0.0704
1,810.8486,1610032.2983,1268.8705,0.9826,0.0950,0.0748
2,901.0224,2448794.0849,1564.8623,0.9792,0.0907,0.0712
3,848.4694,2419126.7919,1555.3542,0.9758,0.0934,0.0695
4,1050.1278,7002519.0515,2646.2273,0.9330,0.1037,0.0789
5,994.0563,6696958.3855,2587.8482,0.9448,0.0990,0.0723
6,938.7380,4533073.7783,2129.1016,0.9560,0.1008,0.0766
7,881.9795,5697680.9620,2386.9816,0.9386,0.0992,0.0745
8,928.9357,2535956.9188,1592.4688,0.9755,0.1003,0.0774


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [9]:
# --- Random Forest Regressor

rf_model = regression.create_model('rf')
rf_model_tuned = regression.tune_model(rf_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,656.5914,1314274.6570,1146.4182,0.9851,0.0717,0.0554
1,609.0533,1119458.5872,1058.0447,0.9879,0.0741,0.0548
2,726.1218,1624885.2422,1274.7099,0.9862,0.0747,0.0573
3,773.2848,1922794.5468,1386.6487,0.9808,0.0822,0.0610
4,846.1857,3810255.1626,1951.9875,0.9635,0.0808,0.0617
5,896.0112,5617760.7320,2370.1816,0.9537,0.0874,0.0615
6,801.7685,3874840.9475,1968.4616,0.9624,0.0833,0.0615
7,702.3567,2556332.8830,1598.8536,0.9724,0.0772,0.0588
8,803.4151,2040523.4085,1428.4689,0.9803,0.0808,0.0604


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,803.0107,2236428.7769,1495.4694,0.9746,0.0900,0.0687
1,749.1237,1297440.7356,1139.0526,0.9860,0.0916,0.0711
2,901.1614,2521887.2106,1588.0451,0.9786,0.0941,0.0722
3,897.0724,2688493.1703,1639.6625,0.9731,0.0985,0.0718
4,1054.8961,6336389.1464,2517.2185,0.9394,0.1037,0.0800
5,1028.1158,7024226.8276,2650.3258,0.9421,0.1031,0.0730
6,975.1932,4218399.0310,2053.8742,0.9591,0.1037,0.0775
7,932.5143,5456801.8410,2335.9798,0.9412,0.1059,0.0791
8,949.6886,3002411.2887,1732.7467,0.9710,0.1060,0.0771


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [10]:
# --- Gradient Boosting Regressor

gbr_model = regression.create_model('gbr')
gbr_model_tuned = regression.tune_model(gbr_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,860.5037,2117421.9211,1455.1364,0.9760,0.1030,0.0819
1,806.7037,2565487.5621,1601.7139,0.9723,0.0969,0.0715
2,1047.1675,3301749.2700,1817.0716,0.9720,0.1033,0.0812
3,885.7624,2184437.4004,1477.9842,0.9782,0.1050,0.0792
4,985.2669,5901222.9487,2429.2433,0.9435,0.1064,0.0792
5,1075.7090,6140920.6310,2478.0881,0.9494,0.1020,0.0760
6,976.6967,3523560.4822,1877.1149,0.9658,0.1109,0.0875
7,888.8486,2732049.6972,1652.8913,0.9705,0.0998,0.0770
8,950.4537,3387457.8659,1840.5048,0.9673,0.1004,0.0745


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,681.5060,1866890.9087,1366.3422,0.9788,0.0748,0.0565
1,757.7254,2141894.2197,1463.5212,0.9768,0.0843,0.0614
2,859.1400,2600986.3636,1612.7574,0.9779,0.0825,0.0610
3,790.5222,2213956.8765,1487.9371,0.9779,0.0934,0.0652
4,811.7979,2384339.0177,1544.1305,0.9772,0.0837,0.0629
5,902.6545,4409845.8606,2099.9633,0.9636,0.0846,0.0613
6,877.2242,3407617.7150,1845.9734,0.9669,0.0890,0.0702
7,675.6107,1575508.9002,1255.1928,0.9830,0.0741,0.0565
8,850.0109,2502146.9952,1581.8176,0.9758,0.0802,0.0610


Fitting 10 folds for each of 10 candidates, totalling 100 fits
